### Reference
- [Build your own Grounded RAG application using Vertex AI APIs for RAG and Langchain](https://github.com/GoogleCloudPlatform/applied-ai-engineering-samples/blob/main/genai-on-vertex-ai/retrieval_augmented_generation/diy_rag_with_vertexai_apis/build_grounded_rag_app_with_vertex.ipynb)

In [2]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_google_community import VertexAIRank, VertexAISearchRetriever, VertexAICheckGroundingWrapper
from dotenv import load_dotenv
import os
from typing import List
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.docstore.document import Document
from langchain_core.runnables import chain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate
from util.helper import get_sxs_comparison, display_grounded_generation
from rich import print

In [3]:
load_dotenv()

True

In [4]:
PROJECT_ID=os.environ.get("PROJECT_ID")
LOCATION=os.environ.get("LOCATION")
COLLECTION=os.environ.get("COLLECTION")
DATA_STORE_ID=os.environ.get("DATA_STORE_ID")
APP_ENGINE_DISPLAY_NAME=os.environ.get("APP_ENGINE_DISPLAY_NAME")
APP_ENGINE_ID=os.environ.get("APP_ENGINE_ID")
FIELD="file_name"
RERANK_TOP_K=5
query="How does Google Cloud leverage AI to enhance its products and services, and what is its commitment to privacy and security in this context?"

In [5]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    data_store_id=DATA_STORE_ID,
    location_id=LOCATION,
    engine_data_type=0,
    max_documents=10,
    max_extractive_segment_count=5
)

In [6]:
# Instantiate the VertexAIReranker with the SDK manager
reranker = VertexAIRank(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    ranking_config="default_ranking_config",
    title_field=FIELD,  # metadata field to preserve with reranked results
    top_n=RERANK_TOP_K,
)


# Create the ContextualCompressionRetriever with the VertexAIRanker as a Reranker
retriever_with_reranker = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=retriever
)

In [9]:
reranked_results = get_sxs_comparison(
    simple_retriever=retriever,
    reranking_api_retriever=retriever_with_reranker,
    query="What were alphabet revenues in 2021?",
    search_kwargs={"k": RERANK_TOP_K},
)

Retriever Results,Reranked Results


In [10]:
llm = VertexAI(model_name="gemini-1.5-pro-001", max_output_tokens=1024)
template = """
Answer the question based only on the following context:
{context}

Question:
{query}
"""
prompt = PromptTemplate.from_template(template)

create_answer = prompt | llm

In [11]:
output_parser = VertexAICheckGroundingWrapper(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    grounding_config="default_grounding_config",
    top_n=3,
)

In [12]:
@chain
def check_grounding_output_parser(answer_candidate: str, documents: List[Document]):
    return output_parser.with_config(configurable={"documents": documents}).invoke(
        answer_candidate
    )


setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)


@chain
def qa_with_check_grounding(query):
    docs = setup_and_retrieval.invoke(query)
    # The GroundingFact attribute dictionary in the grounding API only accepts values that are strings, and it limits the number of attributes to six.
    for doc in docs["context"]:
        doc.metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, str) and k not in ["file_name_2", "created_unix_time"] }
    answer_candidate = create_answer.invoke(input={"query": query, "context": docs["context"]})
    check_grounding_output = check_grounding_output_parser.invoke(
        answer_candidate, documents=docs["context"]
    )
    return check_grounding_output

In [13]:
result = qa_with_check_grounding.invoke(query)
print(result)

CheckGroundingResponse(
    support_score=0.5781737565994263,
    cited_chunks=[
        {
            'chunk_text': "Google Cloud continues to build Al into numerous solutions that our customers can use to
develop Al-powered applications — including processing documents, images, and translation to understand and analyze
data more efficiently, and to use packaged solutions for a variety of industries. In all these examples, Al 
significantly enhances the usefulness and multiplies the value of these products and services to people and 
organizations. Our view is that Al is now, and more than ever, critical to delivering on our mission. As we bring 
our breakthrough Al innovations into the real world to assist people and benefit society everywhere, we are also 
pursuing further advancements that will help to unlock scientific discoveries and to tackle humanity's greatest 
challenges and opportunities.\n\n# Privacy and security\n\nWe make it a priority to protect the privacy and 
security of our products, users, and customers, even if there are near-term financial consequences. We do this by 
continuously investing in building products that are secure by default; strictly upholding responsible data 
practices that emphasize privacy by design; and building easy-to-use settings that put people in control.",
            'source': Document(
                metadata={
                    'id': 'doc-1',
                    'source': 'gs://<bucket>/abehsu/devfest/files/20230203_alphabet_10K.pdf',
                    'company': 'google',
                    'doc_type': 'public',
                    'file_type': 'pdf',
                    'file_name': '20230203-alphabet-10K.pdf'
                },
                page_content="# The power of Al\n\nWe believe that Al is a foundational and transformational 
technology that will provide compelling and helpful benefits to people and society through its capacity to assist, 
complement, empower, and inspire people in almost every field of human endeavor. As an information and computer 
science company, we will continue to be at the forefront of advancing the frontier of Al. Through our path-breaking
and field-defining research and development, we responsibly and boldly develop more capable and useful Al every 
day. Al already powers Google's core products that help billions of people every day and has been at the foundation
of our core ads quality systems for years, helping large and small businesses all over the world to produce and run
effective and efficient ad campaigns that help grow their businesses. Al makes it possible to search in new 
languages, with multiple inputs, such as using images and text at the same time with the Google App. Some of our 
most popular products at Google - including Lens and Translate were built entirely using artificial intelligence 
technologies such as optical character recognition and machine learning. Google Cloud continues to build Al into 
numerous solutions that our customers can use to develop Al-powered applications — including processing documents, 
images, and translation to understand and analyze data more efficiently, and to use packaged solutions for a 
variety of industries. In all these examples, Al significantly enhances the usefulness and multiplies the value of 
these products and services to people and organizations. Our view is that Al is now, and more than ever, critical 
to delivering on our mission. As we bring our breakthrough Al innovations into the real world to assist people and 
benefit society everywhere, we are also pursuing further advancements that will help to unlock scientific 
discoveries and to tackle humanity's greatest challenges and opportunities.\n\n# Privacy and security\n\nWe make it
a priority to protect the privacy and security of our products, users, and customers, even if there are near-term 
financial consequences. We do this by continuously investing in building products that are secure by default; 
strictly uph

In [14]:
display_grounded_generation(result)

In [15]:
result = qa_with_check_grounding.invoke(
    query
)
display_grounded_generation(result)